# Storm Identification and Mode Classification 

In [1]:
import sys, os 
current_dir = os.getcwd()
path = os.path.dirname(current_dir)
sys.path.append(path)

In [2]:
import monte_python

import pickle
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from matplotlib.colors import ListedColormap
cmap = ListedColormap(["white", "red", "blue", "green", "purple", 'gray'])

In [3]:
mode_converter = {
        "NONROT": "ORD",
        "ROTATE": "SUP",
        "QLCS": "QLCS",
        "SUP_CLUST": "SUP_CLUS",
        "QLCS_NON": "QLCS_ORD",
        "QLCS_ROT": "QLCS_MESO",
        "CLUSTER": "CLUS",
        "AMORPHOUS": "OTHER",
        "CLUS_NON": "ORD",
        "CLUS_ROT": "SUP",
        "CLUS_CLU": "CLUS_CLUUUUUUUUU",
        "CELL_ROT": "CELL_ROTTTTTTTTTT",
        "CELL_NON": "CELL_ORDDDDDDDDDD",
    }

retain_storm_types = [
    "ROTATE",
    "NONROT",
    "SEGMENT",
    "QLCS",
    "CLUSTER",
    "LIN_CLUS",
    "AMORPHOUS",
    "BOW_QLCS",
    "SUP_CLUST",
]

storm_type_digitize = {key: n for n, key in enumerate(retain_storm_types)}

In [4]:
with open('/Users/monte.flora/Desktop/storm_mode/STORM_OBJECTS_newse1.pkl', 'rb') as f:
    data = pickle.load(f)

In [5]:
dbz_vals = data['values'][0]
rot_vals = data['rot_values'][0]

In [6]:
dbz_qc_params = [
            ("min_area", 15),
            ("merge_thresh", 3),
            ("max_thresh", (44.93, 100)),
        ]

dbz_qc_params2 = [
            ("min_area", 10),
            ("merge_thresh", 0),
            ("max_thresh", (44.93, 100)),
        ]

uh_qc_params = [("merge_thresh", 1),
                ("min_area", 5),
                ]

clf = monte_python.StormModeClassifier(dbz_thresh=43.0, 
                                       rot_thresh=55,
                                       dbz_qc_params = dbz_qc_params,
                                       rot_qc_params = uh_qc_params,
                                       dbz_qc_params_emb = dbz_qc_params2,
                                       verbose=True
                                      )

results = clf.classify(dbz_vals = dbz_vals, rot_vals=rot_vals, label_embedded=False)
storm_labels, object_props = results['dbz_labels'], results['dbz_props']
#storm_modes, storm_embs = results['storm_modes'], results['storm_embs']

qc_params [('min_area', 15), ('merge_thresh', 3), ('max_thresh', (44.93, 100))]
min_area
merge_thresh
max_thresh
qc_params [('merge_thresh', 1), ('min_area', 5)]
merge_thresh
min_area
self.grid_spacing 3000
area: 2.16e+08, Leng: 31515.66, ecc:0.954
self.grid_spacing 3000
area: 3.33e+08, Leng: 35532.72, ecc:0.929
self.grid_spacing 3000
area: 2.97e+08, Leng: 25230.89, ecc:0.775
self.grid_spacing 3000
area: 9.00e+06, Leng: 0.00, ecc:0.000
self.grid_spacing 3000
area: 4.50e+08, Leng: 34775.16, ecc:0.839
self.grid_spacing 3000
area: 1.98e+08, Leng: 17679.28, ecc:0.583
self.grid_spacing 3000
area: 4.50e+07, Leng: 9295.16, ecc:0.816
self.grid_spacing 3000
area: 3.69e+08, Leng: 42077.51, ecc:0.939
self.grid_spacing 3000
area: 7.20e+07, Leng: 12361.21, ecc:0.812
self.grid_spacing 3000
area: 9.00e+07, Leng: 13145.34, ecc:0.791
self.grid_spacing 3000
area: 1.80e+07, Leng: 6000.00, ecc:1.000
self.grid_spacing 3000
area: 3.60e+08, Leng: 42710.56, ecc:0.949
self.grid_spacing 3000
area: 9.90e+07, Len

In [7]:
n = 15*(3000*3000)
print(f'{n:.02e}')

1.35e+08


In [8]:
print(storm_modes)

NameError: name 'storm_modes' is not defined

In [ ]:
mode_converter = {
        "NONROT": "ORD",
        "ROTATE": "SUP",
        "QLCS": "QLCS",
        "SUP_CLUST": "SUP_CLUS",
        "QLCS_NON": "QLCS_ORD",
        "QLCS_ROT": "QLCS_MESO",
        "CLUSTER": "CLUS",
        "AMORPHOUS": "OTHER",
        "CLUS_NON": "ORD",
        "CLUS_ROT": "SUP",
        "CLUS_CLU": "CLUS_CLUUUUUUUUU",
        "CELL_ROT": "CELL_ROTTTTTTTTTT",
        "CELL_NON": "CELL_ORDDDDDDDDDD",
    }

retain_storm_types = [
    "ROTATE",
    "NONROT",
    "SEGMENT",
    "QLCS",
    "CLUSTER",
    "LIN_CLUS",
    "AMORPHOUS",
    "BOW_QLCS",
    "SUP_CLUST",
]


In [ ]:
converted_modes = [mode_converter[m] for m in storm_modes]
print(converted_modes)

In [ ]:
len(np.unique(storm_labels)), len(converted_modes)

In [ ]:
#true_storm_types = data['storm_types'][0]
#print('True Storm Modes: ', true_storm_types, len([int(m) if (m != ' ') else int(m) for m in true_storm_types]) )

In [ ]:
true_labels = data['labels'][0]
print(len(np.unique(true_labels)))
print(len(data['storm_types'][0]))

In [ ]:
np.unique(storm_labels)

In [ ]:
def label_centroid(ax, object_props):
    """Place object label on object's centroid"""
    for region in object_props:
        x_cent,y_cent = region.centroid
        x_cent=int(x_cent)
        y_cent=int(y_cent)
        xx, yy = x[x_cent,y_cent], y[x_cent,y_cent]
        fontsize = 6.5 if region.label >= 10 else 8
        ax.text(xx,yy,
                    region.label,
                    fontsize=fontsize,
                    ha='center',
                    va='center',
                    color = 'k'
                    )

x,y = np.meshgrid(range(250), range(250))
fig, axes = plt.subplots(dpi=300, ncols=2, figsize=(8,4))

true_labels = data['labels'][0]

for ax, z, cm in zip(axes, [dbz_vals, storm_labels], ['jet', cmap]):
    ax.pcolormesh(x, y, z, cmap=cm)
#label_centroid(ax, object_props) 